In [5]:
## please delete below code after convertion in converted script(py) file
## + 필요없는 내용 삭제(초반부 1,3~14열, In[ ]형태의 주석제거)
!jupyter nbconvert --to script search_cells.ipynb
!sed -i '/^#[ ]In\[/d' search_cells.py
!sed -i -e '1d;3,14d' search_cells.py

[NbConvertApp] Converting notebook search_cells.ipynb to script
[NbConvertApp] Writing 2313 bytes to search_cells.py


In [ ]:
import torch
import torch.nn as nn
from models import ops

In [ ]:
class SearchCell(nn.Module):
    """
    Cell
    Each edge is mixed continuous relaxed.
    """
    def __init__(self, n_nodes, C_pp, C_p, C, reduction_p, reduction):
        """
        Args:
            n_nodes: # of intermediate n_nodes
            C_pp: C_out[k-2]
            C_p : C_out[k-1]
            C   : C_in[k] (current)
            reduction_p: flag for whether the previous cell is reduction cell or not
            reduction: flag for whether the current cell is reduction cell or not
        """
        super().__init__()
        self.reduction = reduction
        self.n_nodes = n_nodes
        
        # If previous cell is reduction cell, current input size does not match with output size of cell[k-2].
        # So the output[k-2] should be reduced by preprocessing.
        if reduction_p:
            self.preproc0 = ops.FactorizedReduce(C_pp, C, affine=False)
        else:
            self.preproc0 = ops.StdConv(C_pp, C, 1, 1, 0, affine=False)
        self.preproc1 = ops.StdConv(C_p, C, 1, 1, 0, affine=False)
        
        #generate dag
        self.dag = nn.ModuleList()
        for i in range(self.n_nodes):
            self.dag.append(nn.ModuleList())
            for j in range(2+i): #include 2 input nodes
                #reduction should be used only for input node
                stride = 2 if reduction and j < 2 else 1
                op = ops.MixedOp(C, stride)
                self.dag[i].append(op)
                
    def forward(self, s0, s1, w_dag):
        s0 = self.preproc0(s0)
        s1 = self.preproc1(s1)
        
        states = [s0, s1]
        for edges, w_list in zip(self.dag, w_dag):
            s_cur = sum(edges[i](s,w) for i, (s,w) in enumerate(zip(states, w_list)))
            states.append(s_cur)
            
        s_out = torch.cat(states[2:], dim=1)
        return s_out